In [1]:
import numpy as np
import pandas as pd
import random
import pickle


# Importing Data and Labels

In [2]:
chromosomeFile = "Data//matchesList.txt"
labelsFile = "Data//labelList.txt"

with open(chromosomeFile, 'rb') as f:
    matchList = pickle.load(f)

with open(labelsFile, 'rb') as f:
    labels = pickle.load(f)

for x in matchList:
    del x[8]
    del x[0]

# Importing Data and Labels

In [3]:
def generatePopulation(size):
    population = []
    for i in range(size):
        temp = []
        for j in range(0,14):
            temp.append(random.randint(0,256))
        population.append(temp)
    return population

In [4]:
def getExpr():
    expr = ["<expr> + <expr>", "<expr> - <expr>", "<expr> * <expr>", "<expr> / <expr>"]
    for i in range(0,7):
            expr.append("t1Stats[" + str(i) + "]")
    return expr

In [5]:
def formGrammar(chromosome,expr):
    ex = "<expr>"
    grammar = [ex]
    for c in range(0,len(chromosome)):
        for i in range(0,len(grammar)):
            if(grammar[i]==ex):
                addToGrammar = expr[chromosome[c]%len(expr)].split(" ")
                for x in addToGrammar:
                    grammar.insert(i+1,x)
                del grammar[i]
                break
    #t1 = grammar.copy()
    #t2 = grammar.copy()
    #for i in t2:
        #t2[t2.index(i)] = i.replace("t1","t2")
    return grammar

In [6]:
def formString(grammar):
    string = ""
    for i in grammar:
        string += i + " "
    return string

In [58]:
def evalString(grammarString, eMatchList, eLabels):
    tempMatchList = eMatchList.copy()
    fitness = 0
    t1GrammarString = grammarString
    t2GrammarString = grammarString.replace("t1","t2")
    for i in range(len(eMatchList)):
        try:
            t1Stats = eMatchList.copy()[i][:7]
            t2Stats = eMatchList.copy()[i][7:]
            print(t1Stats,"\n",t2Stats,"\n",t1GrammarString,"\n",t2GrammarString)
        except:
            print("Error in grammar", grammarString)
            return -1
    return fitness

In [30]:
workingChromosome = [14, 240, 188, 104, 14, 163, 138, 196, 7, 205, 192, 36, 111, 38]
expr = getExpr()
workingGrammar = formGrammar(workingChromosome,expr)
workingGrammarString = formString(workingGrammar)
print(workingChromosome)
print(workingGrammarString)

[14, 240, 188, 104, 14, 163, 138, 196, 7, 205, 192, 36, 111, 38]
t1Stats[5] / t1Stats[1] - t1Stats[5] / t1Stats[2] 


In [53]:
brokenChromosome = [44, 1, 242, 192, 239, 44, 154, 219, 34, 66, 128, 75, 107, 98]
expr = getExpr()
brokenGrammar = formGrammar(brokenChromosome,expr)
brokenGrammarString = formString(brokenGrammar)
print(brokenChromosome)
print(brokenGrammarString)

[44, 1, 242, 192, 239, 44, 154, 219, 34, 66, 128, 75, 107, 98]
t1Stats[1] + t1Stats[4] - t1Stats[6] + t1Stats[3] + t1Stats[5] - t1Stats[4] + t1Stats[6] + <expr> 


In [59]:
evalString(workingGrammarString, matchList, labels)

[72.95, 2.08, 0.84, 1.7595150001396083, 1.2055085503594263, 1.7105263157894737, 1.0526315789473684] 
 [59.95, 1.65, 1.04, 1.2670581879486378, 1.4680384727023346, 1.4473684210526316, 1.263157894736842] 
 t1Stats[5] / t1Stats[1] - t1Stats[5] / t1Stats[2]  
 t2Stats[5] / t2Stats[1] - t2Stats[5] / t2Stats[2] 
[72.04, 2.24, 0.99, 2.007377610076219, 1.5583540254248012, 1.736842105263158, 1.3421052631578947] 
 [52.81, 1.54, 1.24, 1.2192286980877582, 1.685382347737782, 1.2894736842105263, 1.5] 
 t1Stats[5] / t1Stats[1] - t1Stats[5] / t1Stats[2]  
 t2Stats[5] / t2Stats[1] - t2Stats[5] / t2Stats[2] 
[53.7, 1.39, 1.05, 1.1138023652969615, 1.4594345791265368, 0.9736842105263158, 1.394736842105263] 
 [85.82, 2.79, 0.65, 2.2576728524076506, 1.0284020803672702, 2.3684210526315788, 0.9473684210526315] 
 t1Stats[5] / t1Stats[1] - t1Stats[5] / t1Stats[2]  
 t2Stats[5] / t2Stats[1] - t2Stats[5] / t2Stats[2] 
[71.73, 2.04, 0.86, 1.8511637453185885, 1.1914980996302085, 2.1578947368421053, 1.052631578947368

0

In [60]:
evalString(brokenGrammarString, matchList, labels)

[72.95, 2.08, 0.84, 1.7595150001396083, 1.2055085503594263, 1.7105263157894737, 1.0526315789473684] 
 [59.95, 1.65, 1.04, 1.2670581879486378, 1.4680384727023346, 1.4473684210526316, 1.263157894736842] 
 t1Stats[1] + t1Stats[4] - t1Stats[6] + t1Stats[3] + t1Stats[5] - t1Stats[4] + t1Stats[6] + <expr>  
 t2Stats[1] + t2Stats[4] - t2Stats[6] + t2Stats[3] + t2Stats[5] - t2Stats[4] + t2Stats[6] + <expr> 
[72.04, 2.24, 0.99, 2.007377610076219, 1.5583540254248012, 1.736842105263158, 1.3421052631578947] 
 [52.81, 1.54, 1.24, 1.2192286980877582, 1.685382347737782, 1.2894736842105263, 1.5] 
 t1Stats[1] + t1Stats[4] - t1Stats[6] + t1Stats[3] + t1Stats[5] - t1Stats[4] + t1Stats[6] + <expr>  
 t2Stats[1] + t2Stats[4] - t2Stats[6] + t2Stats[3] + t2Stats[5] - t2Stats[4] + t2Stats[6] + <expr> 
[53.7, 1.39, 1.05, 1.1138023652969615, 1.4594345791265368, 0.9736842105263158, 1.394736842105263] 
 [85.82, 2.79, 0.65, 2.2576728524076506, 1.0284020803672702, 2.3684210526315788, 0.9473684210526315] 
 t1Stats[1

0